In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_groq import ChatGroq
api_key=os.getenv("GROQ_API_KEY")
llm=ChatGroq(groq_api_key=api_key,model_name="llama-3.1-8b-instant")
llm

In [ ]:
from langchain.messages import(
    AIMessage,HumanMessage,SystemMessage
)

In [ ]:
speech="""
Cristiano Ronaldo dos Santos Aveiro is widely considered one of the greatest footballers of all time, and his journey from a small island in Madeira to the pinnacle of world football is nothing short of extraordinary. Born in Funchal, Portugal, Ronaldo grew up in a humble household, raised by hardworking parents who supported his dream despite financial struggles. His talent was visible from an early age, and by the time he joined Sporting CP’s academy, coaches realized he possessed a rare combination of raw talent, athleticism, and determination. At just 12 years old, he left home to pursue football—a difficult decision that shaped his discipline and resilience.

His breakthrough came in 2003 when Manchester United signed him after a preseason match in which he dominated experienced defenders. Under Sir Alex Ferguson's mentorship, Ronaldo evolved from a flashy winger into a complete attacker. He helped the club win three Premier League titles, the 2008 Champions League, and earned his first Ballon d'Or. His ability to reinvent himself—adding physical strength, aerial power, and lethal finishing—set him apart from his peers.

In 2009, Ronaldo transferred to Real Madrid for a world-record fee, beginning a historic era. During his nine years in Spain, he scored 450 goals in 438 games, becoming the club’s all-time top scorer. His rivalry with Lionel Messi fueled debates worldwide, and together they redefined the modern game. Ronaldo won four more Champions League titles at Madrid, including three consecutive championships from 2016 to 2018. His performances in crucial matches, particularly Champions League knockout rounds, built his reputation as one of the most clutch players ever.

After Madrid, he moved to Juventus, where he continued breaking records and played a pivotal role in winning Serie A titles. His dedication to fitness allowed him to remain at elite levels even into his late 30s. For Portugal, Ronaldo became the nation’s all-time top scorer and led them to victories in Euro 2016 and the UEFA Nations League. Beyond football, his influence extends globally—he is among the most followed athletes, a successful entrepreneur, and a symbol of discipline, ambition, and longevity.

"""

In [ ]:
speech

In [ ]:
chat_message=[
    SystemMessage(content="You are expertise in summarizing speeched"),
    HumanMessage(content=f"Please provide a short and concise summary of following speech:\n Text:{speech}") 
]

In [ ]:
llm.get_num_tokens(speech)

In [ ]:
llm.invoke(chat_message).content


## Prompt Template Text Summarization

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

generictemplate = """
Write a summary of the following speech:
Speech: {speech}

Translate the precise summary into {language}.
"""

prompt = ChatPromptTemplate.from_template(generictemplate)
prompt


In [ ]:
complete_prompt=prompt.format(speech=speech,language="Hindi")

In [ ]:
llm.get_num_tokens(complete_prompt)

In [ ]:
# Modern approach - chain the prompt with the LLM directly
chain = prompt | llm

# Run the chain
summary = chain.invoke({'speech': speech, 'language': 'Hindi'})

# If you want just the text content
summary.content

## Stuff Document Chain Text summarization

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader=PyPDFLoader("apjspeech.pdf")
docs=loader.load_and_split()
docs

In [ ]:
template= """ Write a consise andshort summary of the following speech,
Speech:{text}

"""
prompt=ChatPromptTemplate.from_template(template)


In [ ]:
# Modern replacement for load_summarize_chain(..., chain_type="stuff")
chain = prompt | llm

full_text = "\n\n".join([d.page_content for d in docs])

output_summary = chain.invoke({"text": full_text})
output_summary.content


## Map reduce to summarize large documents

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
docs

In [ ]:
final_documents=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=100).split_documents(docs)
final_documents


In [ ]:
len(final_documents)

In [ ]:
chunk_prompts=("""
Please summarize the below speech:
Speech:'{text}'
Summary:
""")

prompt=ChatPromptTemplate.from_template(chunk_prompts)

map_chain=prompt|llm

In [ ]:

final_prompt = ChatPromptTemplate.from_template("""
Provide a final summary of the entire speech using the following key points.
Add a motivational title.
Start with a short introduction.
Then provide the final summary in numbered points.

Speech: {text}
""")


In [ ]:
map_summaries = []

for i, doc in enumerate(final_documents, start=1):
    result = map_chain.invoke({"text": doc.page_content})
    print(f"Chunk {i} summarized")
    map_summaries.append(result.content)


reduce_chain = final_prompt | llm



In [ ]:
# Combine all chunk summaries and produce the final summary
output = reduce_chain.invoke({
    "text": "\n\n".join(map_summaries)
})

output.content

In [ ]:
map_summaries = []

for i, doc in enumerate(final_documents, start=1):
    result = map_chain.invoke({"text": doc.page_content})
    
    print(f"\n--- Chunk {i} Summary ---")
    print(result.content)          # print the actual summary like your tutor
    
    map_summaries.append(result.content)


## Refine chain summarization

In [ ]:
initial_prompt = ChatPromptTemplate.from_template("""
Create an initial summary of the first chunk of the speech:

Speech Chunk:
{text}

Initial Summary:
""")


In [ ]:
refine_prompt = ChatPromptTemplate.from_template("""
We have an existing summary:

{existing_summary}

Refine this summary using the new speech chunk below.
Add only important new information. Keep the summary concise.

New Speech Chunk:
{text}

Refined Summary:
""")


In [ ]:
initial_chain=initial_prompt|llm
refine_chain=refine_prompt|llm

In [ ]:
refined_summary = None

for i, doc in enumerate(final_documents, start=1):
    chunk = doc.page_content
    
    if i == 1:
        result = initial_chain.invoke({"text": chunk})
        refined_summary = result.content
        print("\n--- Initial Summary ---\n", refined_summary)
    else:
        result = refine_chain.invoke({
            "existing_summary": refined_summary,
            "text": chunk
        })
        refined_summary = result.content
        print(f"\n--- Refined Summary After Chunk {i} ---\n", refined_summary)


In [ ]:
print("\n========== FINAL REFINED SUMMARY ==========\n")
print(refined_summary)
